In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
%time
import pandas as pd
import glob, os

train_target = pd.read_csv('../input/lifeqhack/training_target.csv')
files_train = glob.glob('../input/lifeqhack/train/train/*.csv')
files_test = glob.glob('../input/lifeqhack/test/test/*.csv')
train = pd.concat([pd.read_csv(i).assign(ID=os.path.basename(i)) for i in files_train])
test = pd.concat([pd.read_csv(i).assign(ID=os.path.basename(i)) for i in files_test])
train['ID'] = train['ID'].apply(lambda x: x.split(".")[0])
test['ID'] = test['ID'].apply(lambda x: x.split(".")[0])
train = train.merge(train_target, on = 'ID', how = 'left')
train.head()

In [43]:
train = train.sort_values(by=['rr_interval'])
train.head()

In [ ]:
# test[test["rr_interval"].isin(train["rr_interval"])]
# test[['ms',"rr_interval"]][~ test[['ms',"rr_interval"]].isin(train[['ms',"rr_interval"]])]

In [3]:
agg = train.drop(['target'],axis=1).groupby('ID', as_index = False).agg(
    ['max','min','mean','count','std','median']).reset_index()
columns = ['ID']

# Iterate through the variables names
for var in agg.columns.levels[0]:
    # Skip the id name
    if var != 'ID':
        
        # Iterate through the stat names
        for stat in agg.columns.levels[1][:-1]:
            # Make a new column name for the variable and stat
            columns.append('agg_%s_%s' % (var, stat))
agg.columns = columns
train = train.merge(agg, on = 'ID', how = 'left')
train.head()

In [4]:
agg = test.groupby('ID', as_index = False).agg(
    ['max','min','mean','count','std','median']).reset_index()
columns = ['ID']

# Iterate through the variables names
for var in agg.columns.levels[0]:
    # Skip the id name
    if var != 'ID':
        
        # Iterate through the stat names
        for stat in agg.columns.levels[1][:-1]:
            # Make a new column name for the variable and stat
            columns.append('agg_%s_%s' % (var, stat))
agg.columns = columns
test = test.merge(agg, on = 'ID', how = 'left')
test.head()

In [ ]:
#this may crush the ram
# agg = train.drop(['ID'],axis=1).groupby('rr_interval', as_index = False).agg(
#     ['max','min','mean','count']).reset_index()
# columns = ['rr_interval']

# # Iterate through the variables names
# for var in agg.columns.levels[0]:
#     # Skip the id name
#     if var != 'rr_interval':
        
#         # Iterate through the stat names
#         for stat in agg.columns.levels[1][:-1]:
#             # Make a new column name for the variable and stat
#             columns.append('agg_%s_%s' % (var, stat))
# agg.columns = columns
# train = train.merge(agg, on = 'rr_interval', how = 'left')
# test = test.merge(agg, on = 'rr_interval', how = 'left')
# train.head()

In [6]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score

# train_1 = train[train['target']!=1]
X = train.drop(['ID', 'target'],axis=1)
y = train['target']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.7, random_state=1, 
                                                      shuffle=True) #random_state=0

In [12]:
%%time
catboost = CatBoostClassifier(iterations=10 ,
                              task_type="GPU"
                              ,depth=10) #   task_type="GPU",  bagging_temperature=10 80%
catboost.fit(X_train, y_train)
predcatboost = catboost.predict(X_valid)

In [13]:
from sklearn.metrics import accuracy_score

accuracy_score(y_valid, predcatboost)

In [ ]:
import xgboost as xgb
import lightgbm as lgb


lightgbm = lgb.LGBMClassifier(device = 'gpu')
lightgbm.fit(X_train, y_train)
predlightgbm = lightgbm.predict(X_valid)

xgb = xgb.XGBClassifier(n_estimators=500,
                        max_depth=9,
                        learning_rate=0.05)

xgb.fit(X_train, y_train)
predxgb = xgb.predict(X_valid)
accuracy_score(y_valid, predxgb)

In [14]:
# 0 = 136
# 1=7
# 2=23
test_ = test
test_['target'] = catboost.predict(test_.drop(['ID'],axis=1))
test_ = test_.drop_duplicates(subset=["ID"])
test_[['ID', 'target']].to_csv('johannes_new.csv',index=False)
test_['target'].value_counts()

In [ ]:
# def rules(x):
#     if x['target']==2:
#         return 1
#     else:
#         return 0
    

# test_['target'] = model.predict(test_.drop(['ID'],axis=1))
# # test_['model_2'] = model_2.predict(test_.drop(['ID'],axis=1))
# test_ = test_.drop_duplicates(subset=["ID"])
# test_['target'] = test_.apply(rules, axis=1)
# test_['target'].value_counts()